In [9]:
# Script 1
# Import required Libraries
import matplotlib.pyplot as plt
import pandas as pd
import requests
from scipy.stats import linregress
import json
from datetime import datetime

# Import API keys
from config import geoapify_key
from config import weather_api_key

# Turn off warning messages
import warnings
warnings.filterwarnings("ignore")

In [2]:
# Script 2
# Load HME projects info into a Dataframe
projects_df = pd.read_csv("../project-1/HMEOpenLargeJobs_20231202.csv")

# Display sample data
projects_df.head()

,ProjectID,Project-Number,Name,Address,Geometry,LocationJSON
0,85216,18-054,PAR 1244,"5950 York St, Denver, CO 80216, USA","{""location"":{""lat"":39.805279,""lng"":-104.958759...","{""Street1"":""5950 York St"",""Street2"":null,""City..."
1,87261,18-121,SECURITY SANITATION WWTF IMPROVEMENTS,"6510 Southmoor Dr, Fountain, CO 80817, USA","{""location"":{""lat"":38.736055,""lng"":-104.737949...","{""Street1"":""6510 Southmoor Dr"",""Street2"":null,..."
2,87538,18-145,WESTERN CURRENCY FACILITY EXPANSION,"8776 Blue Mound Rd, Fort Worth, TX 76131, USA","{""location"":{""lat"":32.8943727,""lng"":-97.348633...","{""Street1"":""8776 Blue Mound Rd"",""Street2"":null..."
3,87544,18-147,BID PACKAGE 4.3 - LEONARD WTP YARD PIPE,"361 Co Rd 4965, Leonard, TX 75452, USA","{""location"":{""lat"":33.383391,""lng"":-96.2825639...","{""Street1"":""361 Co Rd 4965"",""Street2"":null,""Ci..."
4,88609,19-040,SADDLE CREEK RTB,"2520 S 64th St, Omaha, NE 68106, USA","{""location"":{""lat"":41.2404302,""lng"":-96.011843...","{""Street1"":""2520 S 64th St"",""Street2"":null,""Ci..."


In [3]:
# Script 3

# Load the data into a DataFrame (assuming the CSV file is in the same directory as the script)
projects_df = pd.read_csv("HMEOpenLargeJobs_20231202.csv")

# Define a function to parse the 'Geometry' column for latitude and longitude
def extract_lat_lng(geometry_str):
    try:
        geometry = json.loads(geometry_str)
        return geometry['location']['lat'], geometry['location']['lng']
    except (json.JSONDecodeError, KeyError):
        return None, None

# Apply the function to extract 'Lat' and 'Lng'
projects_df[['Lat', 'Lng']] = projects_df['Geometry'].apply(lambda x: pd.Series(extract_lat_lng(x)))

# Select the required columns and add empty columns
projects_clean_df = projects_df[['Project-Number', 'Name', 'Address', 'Lat', 'Lng']].copy()
projects_clean_df['Weather_Category'] = ''
projects_clean_df['Color'] = ''
projects_clean_df['Temperature_F'] = ''
projects_clean_df['Windspeed_mph'] = ''

# Remove rows with invalid or missing Lat and Lng
projects_clean_df.dropna(subset=['Lat', 'Lng'], inplace=True)

# Display the cleaned DataFrame
projects_clean_df.head()


,Project-Number,Name,Address,Lat,Lng,Weather_Category,Color,Temperature_F,Windspeed_mph
0,18-054,PAR 1244,"5950 York St, Denver, CO 80216, USA",39.805279,-104.958759,,,,
1,18-121,SECURITY SANITATION WWTF IMPROVEMENTS,"6510 Southmoor Dr, Fountain, CO 80817, USA",38.736055,-104.737949,,,,
2,18-145,WESTERN CURRENCY FACILITY EXPANSION,"8776 Blue Mound Rd, Fort Worth, TX 76131, USA",32.894373,-97.348633,,,,
3,18-147,BID PACKAGE 4.3 - LEONARD WTP YARD PIPE,"361 Co Rd 4965, Leonard, TX 75452, USA",33.383391,-96.282564,,,,
4,19-040,SADDLE CREEK RTB,"2520 S 64th St, Omaha, NE 68106, USA",41.240430,-96.011843,,,,


In [4]:
# Script 4

# Set weather API base URL
weather_api_base_url = "http://api.openweathermap.org/data/2.5/weather?"


In [6]:
# Script 5

def fetch_weather_data(lat, lng):
    # Building the weather API request URL
    url = f"{weather_api_base_url}lat={lat}&lon={lng}&appid={weather_api_key}&units=imperial"
    
    try:
        response = requests.get(url)
        if response.status_code == 200:
            data = response.json()
            weather_conditions = [weather['main'].lower() for weather in data.get('weather', [])]
            wind_speed = round(data.get('wind', {}).get('speed', 0))
            temp = round(data.get('main', {}).get('temp', 0))

            # Categorize weather conditions
            if 'rain' in weather_conditions and wind_speed > 15:
                return temp, wind_speed, 'Wind and Rain', 'Red'
            elif 'snow' in weather_conditions and wind_speed > 15:
                return temp, wind_speed, 'Wind and Snow', 'Red'
            elif wind_speed > 15:
                return temp, wind_speed, 'High Wind', 'Yellow'
            elif 'rain' in weather_conditions:
                return temp, wind_speed, 'Rain', 'Red'
            elif 'snow' in weather_conditions:
                return temp, wind_speed, 'Snow', 'Red'
            elif temp < 32:
                return temp, wind_speed, 'Cold but Good Weather', 'Blue'
            else:
                return temp, wind_speed, 'Good Weather', 'Green'
        else:
            return None, None, 'Unknown', 'Gray'
    except requests.RequestException:
        return None, None, 'Error', 'Gray'

# Update the DataFrame with weather data
for index, row in projects_clean_df.iterrows():
    temp, wind_speed, weather_category, color = fetch_weather_data(row['Lat'], row['Lng'])
    projects_clean_df.at[index, 'Temperature_F'] = temp
    projects_clean_df.at[index, 'Windspeed_mph'] = wind_speed
    projects_clean_df.at[index, 'Weather_Category'] = weather_category
    projects_clean_df.at[index, 'Color'] = color

# Display the updated DataFrame
#pd.set_option('display.max_rows', None)
projects_clean_df.head()

,Project-Number,Name,Address,Lat,Lng,Weather_Category,Color,Temperature_F,Windspeed_mph
0,18-054,PAR 1244,"5950 York St, Denver, CO 80216, USA",39.805279,-104.958759,High Wind,Yellow,30,33
1,18-121,SECURITY SANITATION WWTF IMPROVEMENTS,"6510 Southmoor Dr, Fountain, CO 80817, USA",38.736055,-104.737949,High Wind,Yellow,29,16
2,18-145,WESTERN CURRENCY FACILITY EXPANSION,"8776 Blue Mound Rd, Fort Worth, TX 76131, USA",32.894373,-97.348633,High Wind,Yellow,41,17
3,18-147,BID PACKAGE 4.3 - LEONARD WTP YARD PIPE,"361 Co Rd 4965, Leonard, TX 75452, USA",33.383391,-96.282564,Good Weather,Green,36,13
4,19-040,SADDLE CREEK RTB,"2520 S 64th St, Omaha, NE 68106, USA",41.240430,-96.011843,Cold but Good Weather,Blue,16,12


In [16]:

# Save the dataframe to a CSV file in the current folder
csv_file_name = "projects_weather_data.csv"
projects_clean_df.to_csv(csv_file_name, index=False)


In [10]:
# Script 8

# Function to fetch 5-day/3-hour forecast data
def fetch_5day_3hour_forecast(lat, lng):
    url = f"http://api.openweathermap.org/data/2.5/forecast?lat={lat}&lon={lng}&appid={weather_api_key}&units=imperial"
    try:
        response = requests.get(url)
        if response.status_code == 200:
            return response.json()
        else:
            print(f"Failed to fetch data: HTTP {response.status_code}")
            return None
    except requests.RequestException as e:
        print(f"Request failed: {e}")
        return None

# Function to check for construction weather days
def check_construction_weather_day(forecast_data):
    construction_weather_days = []
    for entry in forecast_data.get('list', []):
        timestamp = entry['dt']
        date = datetime.fromtimestamp(timestamp)
        temp = entry['main']['temp']
        wind_speed = entry['wind']['speed']
        weather_conditions = [weather['main'].lower() for weather in entry['weather']]
        
        if 'rain' in weather_conditions and (6 <= date.hour <= 18):
            construction_weather_days.append((date.strftime("%Y-%m-%d %H:%M:%S"), 'Rain'))
        elif 'snow' in weather_conditions and (6 <= date.hour <= 18):
            construction_weather_days.append((date.strftime("%Y-%m-%d %H:%M:%S"), 'Snow'))
        elif wind_speed > 15 and (6 <= date.hour <= 18):
            construction_weather_days.append((date.strftime("%Y-%m-%d %H:%M:%S"), 'High Wind'))
        elif temp <= 20 and (6 <= date.hour <= 18):
            construction_weather_days.append((date.strftime("%Y-%m-%d %H:%M:%S"), 'Extreme Low Temperature'))
        elif (6 <= date.hour <= 18):construction_weather_days.append((date.strftime("%Y-%m-%d %H:%M:%S"), 'good weather'))  
           
    return construction_weather_days

# List to store weather data
weather_data_list = []

# Populate the list with weather data
for _, project in projects_clean_df.iterrows():
    forecast_data = fetch_5day_3hour_forecast(project['Lat'], project['Lng'])
    if forecast_data:
        construction_weather_days = check_construction_weather_day(forecast_data)
        for weather_date, weather_type in construction_weather_days:
            weather_data_list.append({
                'Project-Number': project['Project-Number'],
                'Name': project['Name'],
                'Lat': project['Lat'],
                'Lng': project['Lng'],
                'Weather_Date': weather_date,
                'Weather_Type': weather_type
            })
    else:
        print(f"No data for project {project['Project-Number']}")

# Create DataFrame from the list
Weather_5Day_Forecast = pd.DataFrame(weather_data_list)

# Display the DataFrame
Weather_5Day_Forecast.head()

,Project-Number,Name,Lat,Lng,Weather_Date,Weather_Type
0,18-054,PAR 1244,39.805279,-104.958759,2024-01-10 09:00:00,High Wind
1,18-054,PAR 1244,39.805279,-104.958759,2024-01-10 12:00:00,High Wind
2,18-054,PAR 1244,39.805279,-104.958759,2024-01-10 15:00:00,High Wind
3,18-054,PAR 1244,39.805279,-104.958759,2024-01-10 18:00:00,good weather
4,18-054,PAR 1244,39.805279,-104.958759,2024-01-11 06:00:00,good weather


In [9]:

# Save the dataframe to a CSV file in the current folder
csv_file_name_8a = "Weather_5Day_Forecast.csv"
Weather_5Day_Forecast.to_csv(csv_file_name_8a, index=False)
